## Calculate methyl-impact column and add it to the look-up table

In [1]:
import pandas as pd
from pathlib import Path

mu_tbl = pd.read_table(Path('mutation_rate_by_context_methyl.txt'), sep='\t')
mu_tbl['methyl_factor'] = mu_tbl['mu'].div(mu_tbl.groupby(['context','ref','alt'])['mu'].transform('mean'))
mu_tbl 

,context,ref,alt,methylation_level,possible,observed,proportion_observed,mu,fitted_po,methyl_factor
0,AAA,A,C,0,158078695,4892877,0.030952,1.797200e-09,0.033380,1.00000
1,AAA,A,G,0,157804888,8303442,0.052618,3.005400e-09,0.055145,1.00000
2,AAA,A,T,0,158205344,2775294,0.017542,1.040400e-09,0.019492,1.00000
3,AAC,A,C,0,60483349,1497979,0.024767,1.415600e-09,0.026402,1.00000
4,AAC,A,G,0,60298107,5705096,0.094615,5.260400e-09,0.094466,1.00000
...,...,...,...,...,...,...,...,...,...,...
151,TCG,C,T,14,29681,28553,0.961996,1.653100e-07,0.955671,1.57134
152,TCG,C,T,15,13148,12661,0.962960,1.689300e-07,0.958595,1.60575
153,TCT,C,A,0,90590917,4398076,0.048549,2.630300e-09,0.048441,1.00000
154,TCT,C,G,0,90724180,5960824,0.065703,3.556200e-09,0.064905,1.00000


In [3]:
mu_tbl.to_csv('methyl_lookup.txt', sep='\t', index=False)

## Analyze the main code

- `run_nc_constraint_gnomad_v31_main.py` 
1. Prefilter context and genome hail tables. (38 - 43)
    - Input: `context_prepared.ht`, and `genome_prepared.ht`
    - Output: `context_prefiltered.ht` and `genome_prefiltered.ht`
    - `remove_coverage_outliers`, `filter_to_autosomes_par`
2. Compute $\mu$ for each context and methy level from downsampled data. (48 - 96)
    - Input: `context_downsampled_1000.ht`, and `genome_downsampled_1000.ht`
    - Output: `mu_by_context_methyl_downsampled_1000.txt`
    - Counts by groupinng context, ref, alt, methylation level. `count_variants` does the job to generate `observed_counts_by_context_methyl_downsampled_1000.ht`
    - Generic group-by and aggregator to generate `possible_counts_by_context_methyl_downsampled_1000.txt`
3. Compute proportion of possible observed for each context and methyl level from the whole dataset. (101 - 127)
    - Input: `context_prefiltered.ht` (1 output), and `genome_prefiltered.ht` (1 output)
    - Output: `observed_counts_by_context_methyl`, and `possible_counts_by_context_methyl.txt`
    - Another round of grouping and counting but with tables joined this time
4. Fit proportion_observed ~ $\mu$ to obtain the context-specific mutabilities. (132 - 167)
    - Input: `possible_counts_by_context_methyl.txt` (3 output), `observed_counts_by_context_methyl.txt` (3 output), and `mu_by_context_methyl_downsampled_1000.txt` (2 output)
    - Output: `mutation_rate_by_context_methyl.txt`
    - `np.polyfit`
5. Compute expected number of variants across the genome (per 1kb) based on the context-specific mutation rates. (174 - 215)
    - Input: `context_prefiltered.ht` (1 output), `genome_prefiltered.ht` (1 output), `hg38.chrom.1kb.bed`, `mutation_rate_by_context_methyl.txt` (4 output). (174 - 215)
    - Output: `expected_counts_per_context_methyl_genome_1kb.{ht,txt}`, `observed_counts_genome_1kb.txt`
    - Add a new column `element_id` to both `context_ht` and `genome_ht` from the input bed file to the context and genome hail tables
    - Create `observed_ht` (`observed_counts_genome_1kb.txt`) by counting values of `genome_ht` with group-by (`element_id`)
    - Create `possible_ht` by count values of `context_ht` with group-by (`context`, `ref`, `alt`, `methyl_level`, `element_id`)
    - Add a new column `expected` to `possible_ht` where `expected=possible_ht.variant_count*po_ht[possible_ht.key].fitted_po` and `possible_ht.key=('context','ref','alt','methylation_level')`
    - Create a new hail table `expected_ht` (`expected_counts_per_context_methyl_genome_1kb.ht`) consists of columns by summing up `possible_ht.variant_count` (`possible`) and `possible_ht.expected` (`expected`) with group-by (`element_id`,`context`)
    - observed (from genome), possilbe (from context), and expected (possible and mutation rates)
6. Adjust the effects of regional genomic features on mutation rates, per 1kb
7. Put everything together to run Gnocchi ()

### Genomic features 

In [4]:
import pandas as pd
from pathlib import Path

data_dir = Path('/mnt/isilon/immgen_res/data/gnomad-nc-constraint-v31-paper')

df_ft = pd.read_csv(data_dir/'misc/genomic_features13_genome_1kb.txt', sep='\t')
df_ft_sel_ = pd.read_csv(data_dir/'misc/genomic_features13_sel.txt', sep='\t')

In [5]:
df_ft

,element_id,dist2telo_1k,dist2cent_1k,GC_content_1k,LCR_1k,SINE_1k,LINE_1k,recomb_male_1k,recomb_female_1k,met_sperm_1k,...,LCR_1M,SINE_1M,LINE_1M,recomb_male_1M,recomb_female_1M,met_sperm_1M,Nucleosome_1M,cDNM_maternal_05M_1M,cDNM_paternal_05M_1M,CpG_island_1M
0,chr1-26000-27000,0.010644,49.004544,47.9,0.0,26.7,20.8,0.0,0.0,0.000000,...,18.0107,10.6848,17.1641,0.0,0.0,0.005165,1.10606,0.0,0.0,3.4736
1,chr1-27000-28000,0.011046,49.004142,37.7,0.0,22.1,8.4,0.0,0.0,0.000000,...,18.0107,10.6848,17.1641,0.0,0.0,0.005165,1.10606,0.0,0.0,3.4736
2,chr1-28000-29000,0.011448,49.003741,44.6,2.7,16.6,0.0,0.0,0.0,0.001833,...,18.0107,10.6848,17.1641,0.0,0.0,0.005165,1.10606,0.0,0.0,3.4736
3,chr1-29000-30000,0.011849,49.003339,69.6,0.0,0.0,9.9,0.0,0.0,0.001149,...,18.0107,10.6848,17.1641,0.0,0.0,0.005165,1.10606,0.0,0.0,3.4736
4,chr1-30000-31000,0.012251,49.002937,43.6,10.5,0.0,19.8,0.0,0.0,0.000000,...,18.0107,10.6848,17.1641,0.0,0.0,0.005165,1.10606,0.0,0.0,3.4736
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2874985,chr22-14776000-14777000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2874986,chr22-14777000-14778000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2874987,chr22-14778000-14779000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2874988,chr22-14779000-14780000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
df_ft_sel_

,context,feature,window,coef,se,pval
0,AAA,cDNM_maternal_05M,1k,0.067713,0.007894,9.729159e-18
1,AAA,dist2telo,1k,-0.041536,0.008925,3.261823e-06
2,AAA,recomb_male,1k,0.025908,0.005896,1.111162e-05
3,AAC,SINE,10k,-0.048551,0.012292,7.828334e-05
4,AAC,LCR,1k,0.038081,0.010998,5.353477e-04
...,...,...,...,...,...,...
234,TCT,cDNM_maternal_05M,1k,0.092985,0.006904,2.406517e-41
235,TCT,CpG_island,1k,0.035758,0.006393,2.228002e-08
236,TCT,dist2telo,1k,-0.059602,0.007988,8.566329e-14
237,TCT,recomb_male,10k,0.031489,0.006748,3.065350e-06


In [8]:
len(df_ft_sel_['context'].unique())

32

## Hail tables

In [1]:
import hail as hl
from bokeh.io import show
from pathlib import Path

data_dir = Path('/mnt/isilon/immgen_res/data/gnomad-nc-constraint-v31-paper')
hl.init()

Loading BokehJS ...

SLF4J: Failed to load class "org.slf4j.impl.StaticLoggerBinder".
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#StaticLoggerBinder for further details.
Running on Apache Spark version 3.5.1
SparkUI available at http://10.30.43.158:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.131-11d9b2ff89da
LOGGING: writing to /scr1/users/kimh31/gnomad-nc-constraint/hail-20240625-1326-0.2.131-11d9b2ff89da.log


### Methylation levels of positions 

In [2]:
pos_methyl_ht = hl.read_table('data/pos_methyl_downsampled_1000.ht')
print(pos_methyl_ht.describe())
print(pos_methyl_ht.count())

----------------------------------------
Global fields:
    'grange': array<int32> 
----------------------------------------
Row fields:
    'locus': locus<GRCh38> 
    'context': str 
    'ref': str 
    'alt': str 
    'methyl_level': int32 
    'variant_count': int64 
----------------------------------------
Key: ['locus', 'context', 'ref', 'alt', 'methyl_level']
----------------------------------------
None
7859559322


In [3]:
pos_methyl_ht.show()

SLF4J: Failed to load class "org.slf4j.impl.StaticMDCBinder".
SLF4J: Defaulting to no-operation MDCAdapter implementation.
SLF4J: See http://www.slf4j.org/codes.html#no_static_mdc_binder for further details.


,,,,,
locus,context,ref,alt,methyl_level,variant_count
locus<GRCh38>,str,str,str,int32,int64
chr1:10010,"""ACC""","""C""","""A""",0,1
chr1:10010,"""ACC""","""C""","""G""",0,1
chr1:10010,"""ACC""","""C""","""T""",0,1
chr1:10011,"""CCC""","""C""","""A""",0,1
chr1:10011,"""CCC""","""C""","""G""",0,1
chr1:10011,"""CCC""","""C""","""T""",0,1
chr1:10012,"""CCT""","""C""","""A""",0,1
chr1:10012,"""CCT""","""C""","""G""",0,1


In [4]:
pos_methyl_ht.filter(pos_methyl_ht.methyl_level != 0).show()

,,,,,
locus,context,ref,alt,methyl_level,variant_count
locus<GRCh38>,str,str,str,int32,int64
chr1:10650,"""GCG""","""C""","""T""",1,1
chr1:10651,"""GCG""","""C""","""T""",1,1
chr1:10660,"""GCG""","""C""","""T""",1,1
chr1:10661,"""GCG""","""C""","""T""",1,1
chr1:10665,"""CCG""","""C""","""T""",2,1
chr1:10666,"""GCG""","""C""","""T""",2,1
chr1:10667,"""GCG""","""C""","""T""",1,1
chr1:10668,"""GCG""","""C""","""T""",1,1


In [5]:
pos_methyl_ht.filter(pos_methyl_ht.variant_count > 1).show()
# runnint time: 49m 17.7s

,,,,,
locus,context,ref,alt,methyl_level,variant_count
locus<GRCh38>,str,str,str,int32,int64


### Methylation info function

In [4]:
import hail as hl
import pandas as pd
from pathlib import Path

# hl.init()

def get_methyl_adj(chrom, pos, context, ref, alt, context_ht, methyl_adj_tbl=None):
    data_dir = Path('/mnt/isilon/immgen_res/data/gnomad-nc-constraint-v31-paper')
    if methyl_adj_tbl is None:
        methyl_adj_tbl = pd.read_csv(data_dir/'methyl_lookup.txt', sep='\t')
    
    locus_expr = hl.locus(chrom, pos, reference_genome='GRCh38')
    context_4input = context_ht.filter((context_ht.context == context) & (context_ht.ref == ref) & (context_ht.alt == alt) & (context_ht.locus == locus_expr))

    methyl_level = context_4input.aggregate(hl.agg.collect(context_4input.methyl_level))
    methyl_level = methyl_level[0]
    row_slice = methyl_adj_tbl[(methyl_adj_tbl['context'] == context) & (methyl_adj_tbl['ref'] == ref) & (methyl_adj_tbl['alt'] == alt) & (methyl_adj_tbl['methylation_level'] == methyl_level)]

    row_slice = row_slice[['context', 'ref', 'alt', 'methylation_level', 'methyl_factor']].reset_index()
    row_pos = pd.DataFrame({'chrom': chrom, 'pos':pos}, index=[0])
    
    return pd.concat([row_pos, row_slice], axis=1)


context_ht = hl.read_table(str(data_dir/'context_prefiltered.ht'))
x = get_methyl_adj(chrom = 'chr1', pos = 10666, context = 'GCG', ref = 'C', alt = 'T', context_ht=context_ht)

In [3]:
x

,chrom,pos,index,context,ref,alt,methylation_level,methyl_factor
0,chr1,10666,100,GCG,C,T,2,0.389761


In [6]:
intervals = [hl.parse_locus_interval('chr1:10650-10675', reference_genome='GRCh38')] # or some other way to make a list of
ds_result = hl.filter_intervals(context_ht, intervals)

In [35]:
group_vars = ['context','ref','alt','methyl_level']
xsub = ds_result.select(
    ds_result.context,
    ds_result.ref,
    ds_result.alt,
    ds_result.methyl_level
).to_pandas()
xsub['locus'] = xsub['locus'].astype(str)
xsub[['chrom','pos']] = xsub['locus'].str.split(':', expand=True)
xsub.drop(columns='locus', inplace=True)
xsub = xsub[['chrom','pos','alleles']+group_vars]
xsub

,chrom,pos,alleles,context,ref,alt,methyl_level
0,chr1,10650,"[C, A]",GCG,C,A,0
1,chr1,10650,"[C, G]",GCG,C,G,0
2,chr1,10650,"[C, T]",GCG,C,T,1
3,chr1,10651,"[G, A]",GCG,C,T,1
4,chr1,10651,"[G, C]",GCG,C,G,0
...,...,...,...,...,...,...,...
70,chr1,10673,"[C, G]",GCG,C,G,0
71,chr1,10673,"[C, T]",GCG,C,T,2
72,chr1,10674,"[G, A]",GCG,C,T,2
73,chr1,10674,"[G, C]",GCG,C,G,0


In [32]:
data_dir = Path('/mnt/isilon/immgen_res/data/gnomad-nc-constraint-v31-paper')
methyl_adj_tbl = pd.read_csv(data_dir/'methyl_lookup.txt', sep='\t')
if 'methylation_level' in methyl_adj_tbl.columns:
    methyl_adj_tbl.rename(columns={'methylation_level':'methyl_level'}, inplace=True)

In [38]:
pd.merge(xsub, methyl_adj_tbl[group_vars+['methyl_factor']], how='left', on=group_vars)

,chrom,pos,alleles,context,ref,alt,methyl_level,methyl_factor
0,chr1,10650,"[C, A]",GCG,C,A,0,1.000000
1,chr1,10650,"[C, G]",GCG,C,G,0,1.000000
2,chr1,10650,"[C, T]",GCG,C,T,1,0.261286
3,chr1,10651,"[G, A]",GCG,C,T,1,0.261286
4,chr1,10651,"[G, C]",GCG,C,G,0,1.000000
...,...,...,...,...,...,...,...,...
70,chr1,10673,"[C, G]",GCG,C,G,0,1.000000
71,chr1,10673,"[C, T]",GCG,C,T,2,0.389761
72,chr1,10674,"[G, A]",GCG,C,T,2,0.389761
73,chr1,10674,"[G, C]",GCG,C,G,0,1.000000


In [ ]:
""" 
data_dir = Path('/mnt/isilon/immgen_res/data/gnomad-nc-constraint-v31-paper')
methyl_adj_tbl = pd.read_csv(data_dir/'methyl_lookup.txt', sep='\t')

locus_expr = hl.locus(chrom, pos, reference_genome='GRCh38')
context_4input = context_ht.filter((context_ht.context == context) & (context_ht.ref == ref) & (context_ht.alt == alt) & (context_ht.locus == locus_expr))
context_4input.show()
"""

### Methylation info function using range 

In [1]:
import hail as hl
import pandas as pd
from pathlib import Path

hl.init()

def get_methyl_adj(chrom, start, end, context_ht=None, methyl_adj_tbl=None):
    data_dir = Path('/mnt/isilon/immgen_res/data/gnomad-nc-constraint-v31-paper')
    if methyl_adj_tbl is None:
        methyl_adj_tbl = pd.read_csv(data_dir/'methyl_lookup.txt', sep='\t')
        if 'methylation_level' in methyl_adj_tbl.columns:
            methyl_adj_tbl.rename(columns={'methylation_level':'methyl_level'}, inplace=True)

    if context_ht is None:
        hl.init()
        context_ht = hl.read_table(str(data_dir/'context_prefiltered.ht'))

    intervals = [hl.parse_locus_interval(f'{chrom}:{str(start)}-{str(end)}', reference_genome='GRCh38')]
    context_sub_ht = hl.filter_intervals(context_ht, intervals)
    
    group_vars = ['context','ref','alt','methyl_level']
    if context_sub_ht.count() == 0:
        return pd.DataFrame(columns=['chrom','pos','alleles']+group_vars+['methyl_factor'])

    context_sub_ht = context_sub_ht.select(
        context_sub_ht.context,
        context_sub_ht.ref,
        context_sub_ht.alt,
        context_sub_ht.methyl_level)
    context_sub_df = context_sub_ht.to_pandas()

    context_sub_df['locus'] = context_sub_df['locus'].astype(str)
    context_sub_df[['chrom','pos']] = context_sub_df['locus'].str.split(':', expand=True)
    context_sub_df.drop(columns='locus', inplace=True)
    context_sub_df = context_sub_df[['chrom','pos','alleles']+group_vars]
    
    return pd.merge(context_sub_df, methyl_adj_tbl[group_vars+['methyl_factor']], how='left', on=group_vars)

data_dir = Path('/mnt/isilon/immgen_res/data/gnomad-nc-constraint-v31-paper')
context_ht = hl.read_table(str(data_dir/'context_prefiltered.ht'))

# x = get_methyl_adj('chr1', start=10650, end=10675, context_ht=context_ht)
# x

Loading BokehJS ...

SLF4J: Failed to load class "org.slf4j.impl.StaticLoggerBinder".
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#StaticLoggerBinder for further details.
Running on Apache Spark version 3.5.1
SparkUI available at http://10.30.43.158:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.131-11d9b2ff89da
LOGGING: writing to /scr1/users/kimh31/gnomad-nc-constraint/hail-20240625-1553-0.2.131-11d9b2ff89da.log


In [2]:
x = get_methyl_adj('chr6', start=100000, end=101000, context_ht=context_ht)
x

SLF4J: Failed to load class "org.slf4j.impl.StaticMDCBinder".
SLF4J: Defaulting to no-operation MDCAdapter implementation.
SLF4J: See http://www.slf4j.org/codes.html#no_static_mdc_binder for further details.


,chrom,pos,alleles,context,ref,alt,methyl_level,methyl_factor


In [7]:
import time

starts=[10650, 10675, 10700]
ends=[10674, 10694, 10724]

df = []
for start, end in zip(starts, ends):
    startt = time.time()
    df.append(get_methyl_adj('chr1', start=start, end=end, context_ht=context_ht))
    endt = time.time()
    print(f'Elapsed time: {endt-startt}')

Elapsed time: 1.710728406906128
Elapsed time: 1.7431550025939941
Elapsed time: 1.5843884944915771


In [10]:
df[2]

,chrom,pos,alleles,context,ref,alt,methyl_level,methyl_factor
0,chr1,10700,"[G, A]",CCG,C,T,2,0.442922
1,chr1,10700,"[G, C]",CCG,C,G,0,1.000000
2,chr1,10700,"[G, T]",CCG,C,A,0,1.000000
3,chr1,10701,"[G, A]",GCC,C,T,0,1.000000
4,chr1,10701,"[G, C]",GCC,C,G,0,1.000000
...,...,...,...,...,...,...,...,...
64,chr1,10722,"[C, G]",GCC,C,G,0,1.000000
65,chr1,10722,"[C, T]",GCC,C,T,0,1.000000
66,chr1,10723,"[C, A]",CCG,C,A,0,1.000000
67,chr1,10723,"[C, G]",CCG,C,G,0,1.000000


### Examples of methyl info in context

In [8]:
context_ht.filter((context_ht.context == 'ACG') & (context_ht.alt == 'T') & (context_ht.locus == hl.locus('chr1', 29285, reference_genome='GRCh38'))).show()

,,,,,
locus,alleles,context,ref,alt,methyl_level
locus<GRCh38>,array<str>,str,str,str,int32
chr1:29285,"[""C"",""T""]","""ACG""","""C""","""T""",1


In [10]:
context_ht.filter((context_ht.context == 'ACG') & (context_ht.alt == 'T')).show()

,,,,,
locus,alleles,context,ref,alt,methyl_level
locus<GRCh38>,array<str>,str,str,str,int32
chr1:13216,"[""C"",""T""]","""ACG""","""C""","""T""",6
chr1:13302,"[""C"",""T""]","""ACG""","""C""","""T""",10
chr1:13537,"[""C"",""T""]","""ACG""","""C""","""T""",9
chr1:13645,"[""G"",""A""]","""ACG""","""C""","""T""",6
chr1:13694,"[""G"",""A""]","""ACG""","""C""","""T""",6
chr1:13751,"[""G"",""A""]","""ACG""","""C""","""T""",6
chr1:13828,"[""C"",""T""]","""ACG""","""C""","""T""",9
chr1:14355,"[""G"",""A""]","""ACG""","""C""","""T""",6


In [12]:
context_ht.filter((context_ht.context == 'ACG') & (context_ht.alt == 'T') & (context_ht.locus == hl.locus('chr1', 14487, reference_genome='GRCh38'))).show()

,,,,,
locus,alleles,context,ref,alt,methyl_level
locus<GRCh38>,array<str>,str,str,str,int32
chr1:14487,"[""G"",""A""]","""ACG""","""C""","""T""",6


In [2]:
context_down_tbl = hl.read_table(str(data_dir/'context_downsampled_1000.ht'))
context_down_tbl.count()

7859559322

In [5]:
context_down_tbl.filter((context_down_tbl.context == 'ACG') & (context_down_tbl.alt == 'T')).show(10)

,,,,,,,,
locus,alleles,context,ref,alt,cpg,variant_type,variant_type_model,methyl_level
locus<GRCh38>,array<str>,str,str,str,bool,str,str,int32
chr1:13694,"[""G"",""A""]","""ACG""","""C""","""T""",True,"""CpG""","""ACG""",6
chr1:14487,"[""G"",""A""]","""ACG""","""C""","""T""",True,"""CpG""","""ACG""",6
chr1:14711,"[""G"",""A""]","""ACG""","""C""","""T""",True,"""CpG""","""ACG""",6
chr1:16058,"[""C"",""T""]","""ACG""","""C""","""T""",True,"""CpG""","""ACG""",6
chr1:16243,"[""C"",""T""]","""ACG""","""C""","""T""",True,"""CpG""","""ACG""",6
chr1:17453,"[""G"",""A""]","""ACG""","""C""","""T""",True,"""CpG""","""ACG""",7
chr1:17493,"[""G"",""A""]","""ACG""","""C""","""T""",True,"""CpG""","""ACG""",6
chr1:29285,"[""C"",""T""]","""ACG""","""C""","""T""",True,"""CpG""","""ACG""",1


In [3]:
context_down_tbl.filter((context_down_tbl.context == 'ACG') & (context_down_tbl.alt == 'T') & (context_down_tbl.locus == hl.locus('chr1', 16243, reference_genome='GRCh38'))).show(10)

SLF4J: Failed to load class "org.slf4j.impl.StaticMDCBinder".
SLF4J: Defaulting to no-operation MDCAdapter implementation.
SLF4J: See http://www.slf4j.org/codes.html#no_static_mdc_binder for further details.


,,,,,,,,
locus,alleles,context,ref,alt,cpg,variant_type,variant_type_model,methyl_level
locus<GRCh38>,array<str>,str,str,str,bool,str,str,int32
chr1:1105971,"[""G"",""A""]","""ACG""","""C""","""T""",True,"""CpG""","""ACG""",15


In [6]:
context_down_tbl.filter((context_down_tbl.context == 'ACG') & (context_down_tbl.alt == 'T') & (context_down_tbl.locus == hl.locus('chr1', 16243, reference_genome='GRCh38'))).show(10)

,,,,,,,,
locus,alleles,context,ref,alt,cpg,variant_type,variant_type_model,methyl_level
locus<GRCh38>,array<str>,str,str,str,bool,str,str,int32
chr1:16243,"[""C"",""T""]","""ACG""","""C""","""T""",True,"""CpG""","""ACG""",6


In [7]:
context_down_tbl.filter((context_down_tbl.context == 'ACG') & (context_down_tbl.alt == 'T') & (context_down_tbl.locus == hl.locus('chr1', 29285, reference_genome='GRCh38'))).show(10)

,,,,,,,,
locus,alleles,context,ref,alt,cpg,variant_type,variant_type_model,methyl_level
locus<GRCh38>,array<str>,str,str,str,bool,str,str,int32
chr1:29285,"[""C"",""T""]","""ACG""","""C""","""T""",True,"""CpG""","""ACG""",1


In [4]:
small_context_down_tbl = context_down_tbl.sample(0.01)
small_context_down_tbl.count()

SLF4J: Failed to load class "org.slf4j.impl.StaticMDCBinder".
SLF4J: Defaulting to no-operation MDCAdapter implementation.
SLF4J: See http://www.slf4j.org/codes.html#no_static_mdc_binder for further details.


78605333

### Downsampled data

In [7]:
context_down_tbl = hl.read_table(str(data_dir/'context_downsampled_1000.ht'))
context_down_tbl.describe()

----------------------------------------
Global fields:
    'grange': array<int32> 
----------------------------------------
Row fields:
    'locus': locus<GRCh38> 
    'alleles': array<str> 
    'context': str 
    'ref': str 
    'alt': str 
    'cpg': bool 
    'variant_type': str 
    'variant_type_model': str 
    'methyl_level': int32 
----------------------------------------
Key: ['locus', 'alleles']
----------------------------------------


In [21]:
context_down_tbl.show(30)

,,,,,,,,
locus,alleles,context,ref,alt,cpg,variant_type,variant_type_model,methyl_level
locus<GRCh38>,array<str>,str,str,str,bool,str,str,int32
chr1:10010,"[""C"",""A""]","""ACC""","""C""","""A""",False,"""transversion""","""non-CpG""",0
chr1:10010,"[""C"",""G""]","""ACC""","""C""","""G""",False,"""transversion""","""non-CpG""",0
chr1:10010,"[""C"",""T""]","""ACC""","""C""","""T""",False,"""non-CpG transition""","""non-CpG""",0
chr1:10011,"[""C"",""A""]","""CCC""","""C""","""A""",False,"""transversion""","""non-CpG""",0
chr1:10011,"[""C"",""G""]","""CCC""","""C""","""G""",False,"""transversion""","""non-CpG""",0
chr1:10011,"[""C"",""T""]","""CCC""","""C""","""T""",False,"""non-CpG transition""","""non-CpG""",0
chr1:10012,"[""C"",""A""]","""CCT""","""C""","""A""",False,"""transversion""","""non-CpG""",0
chr1:10012,"[""C"",""G""]","""CCT""","""C""","""G""",False,"""transversion""","""non-CpG""",0


In [19]:
context_down_tbl.count()

7859559322

In [4]:
genome_down_tbl = hl.read_table(str(data_dir/'genome_downsampled_1000.ht'))
genome_down_tbl.describe()

----------------------------------------
Global fields:
    'freq_meta': array<dict<str, str>> 
    'freq_index_dict': dict<str, int32> 
    'faf_index_dict': dict<str, int32> 
    'faf_meta': array<dict<str, str>> 
    'vep_version': str 
    'vep_csq_header': str 
    'dbsnp_version': str 
    'filtering_model': struct {
        model_name: str, 
        score_name: str, 
        snv_cutoff: struct {
            bin: float64, 
            min_score: float64
        }, 
        indel_cutoff: struct {
            bin: float64, 
            min_score: float64
        }, 
        model_id: str, 
        snv_training_variables: array<str>, 
        indel_training_variables: array<str>
    } 
    'age_distribution': struct {
        bin_edges: array<float64>, 
        bin_freq: array<int32>, 
        n_smaller: int32, 
        n_larger: int32
    } 
----------------------------------------
Row fields:
    'locus': locus<GRCh38> 
    'alleles': array<str> 
    'context': str 
    'ref': str

In [5]:
genome_down_tbl.show()

+---------------+------------+---------+-----+-----+
| locus         | alleles    | context | ref | alt |
+---------------+------------+---------+-----+-----+
| locus<GRCh38> | array<str> | str     | str | str |
+---------------+------------+---------+-----+-----+
| chr1:10111    | ["C","A"]  | "ACC"   | "C" | "A" |
| chr1:10126    | ["T","C"]  | "TAG"   | "A" | "G" |
| chr1:10126    | ["T","G"]  | "TAG"   | "A" | "C" |
| chr1:10132    | ["T","C"]  | "TAG"   | "A" | "G" |
| chr1:10139    | ["A","T"]  | "TAA"   | "A" | "T" |
| chr1:10140    | ["A","C"]  | "AAC"   | "A" | "C" |
| chr1:10141    | ["C","G"]  | "ACC"   | "C" | "G" |
| chr1:10149    | ["C","A"]  | "CCC"   | "C" | "A" |
| chr1:10154    | ["C","A"]  | "ACC"   | "C" | "A" |
| chr1:10172    | ["C","G"]  | "ACC"   | "C" | "G" |
+---------------+------------+---------+-----+-----+

+------------------------------------------------------------------------------+
| freq                                                                         |
+------------------------------------------------------------------------------+
| array<struct{AC: int32, AF: float64, AN: int32, homozygote_count: int32}>    |
+------------------------------------------------------------------------------+
| [(1,2.26e-05,44330,0),(2,2.08e-05,96154,0),(0,0.00e+00,22282,0),(1,4.87e-... |
| [(1,2.36e-05,42350,0),(9,1.23e-04,72892,0),(1,4.27e-05,23402,0),(0,0.00e+... |
| [(1,2.36e-05,42354,0),(6,8.23e-05,72892,0),(0,0.00e+00,23408,0),(0,0.00e+... |
| [(10,2.63e-04,38078,0),(14,2.14e-04,65292,0),(6,2.80e-04,21406,0),(0,0.00... |
| [(1,1.70e-05,58738,0),(4,4.06e-05,98452,0),(0,0.00e+00,31104,0),(0,0.00e+... |
| [(2,2.75e-05,72696,0),(12,1.03e-04,116278,0),(2,5.68e-05,35202,0),(0,0.00... |
| [(1,1.97e-05,50842,0),(1,1.03e-05,97002,0),(1,3.75e-05,26636,0),(0,0.00e+... |
| [(1,5.87e-05,17024,0),(3,2.86e-05,104766,0),(1,1.41e-04,7080,0),(0,0.00e+... |
| [(1,3.62e-05,27646,0),(6,5.98e-05,100306,0),(0,0.00e+00,13428,0),(0,0.00e... |
| [(1,2.66e-05,37600,0),(5,5.86e-05,85282,1),(1,4.96e-05,20154,0),(0,0.00e+... |
+------------------------------------------------------------------------------+

+--------------+-------+----------------------+--------------------+
| pass_filters |   cpg | variant_type         | variant_type_model |
+--------------+-------+----------------------+--------------------+
|         bool |  bool | str                  | str                |
+--------------+-------+----------------------+--------------------+
|         True | False | "transversion"       | "non-CpG"          |
|         True | False | "non-CpG transition" | "non-CpG"          |
|         True | False | "transversion"       | "non-CpG"          |
|         True | False | "non-CpG transition" | "non-CpG"          |
|         True | False | "transversion"       | "non-CpG"          |
|         True | False | "transversion"       | "non-CpG"          |
|         True | False | "transversion"       | "non-CpG"          |
|         True | False | "transversion"       | "non-CpG"          |
|         True | False | "transversion"       | "non-CpG"          |
|         True | False | "transversion"       | "non-CpG"          |
+--------------+-------+----------------------+--------------------+

+--------------+
| methyl_level |
+--------------+
|        int32 |
+--------------+
|            0 |
|            0 |
|            0 |
|            0 |
|            0 |
|            0 |
|            0 |
|            0 |
|            0 |
|            0 |
+--------------+
showing top 10 rows

### Full data

In [5]:
context_tbl = hl.read_table(str(data_dir/'context_prefiltered.ht'))
context_tbl.describe()

----------------------------------------
Global fields:
    'grange': array<int32> 
----------------------------------------
Row fields:
    'idx': int32 
    'locus': locus<GRCh38> 
    'alleles': array<str> 
    'a_index': int32 
    'was_split': bool 
    'old_locus': locus<GRCh38> 
    'old_alleles': array<str> 
    'vep': struct {
        assembly_name: str, 
        allele_string: str, 
        ancestral: str, 
        colocated_variants: array<struct {
            aa_allele: str, 
            aa_maf: float64, 
            afr_allele: str, 
            afr_maf: float64, 
            allele_string: str, 
            amr_allele: str, 
            amr_maf: float64, 
            clin_sig: array<str>, 
            end: int32, 
            eas_allele: str, 
            eas_maf: float64, 
            ea_allele: str, 
            ea_maf: float64, 
            eur_allele: str, 
            eur_maf: float64, 
            exac_adj_allele: str, 
            exac_adj_maf: float64, 
          

In [7]:
context_tbl.show(5)

+-------+---------------+------------+---------+-----------+---------------+
|   idx | locus         | alleles    | a_index | was_split | old_locus     |
+-------+---------------+------------+---------+-----------+---------------+
| int32 | locus<GRCh38> | array<str> |   int32 |      bool | locus<GRCh38> |
+-------+---------------+------------+---------+-----------+---------------+
|  1000 | chr1:10010    | ["C","A"]  |       1 |      True | chr1:10010    |
|  1000 | chr1:10010    | ["C","G"]  |       2 |      True | chr1:10010    |
|  1000 | chr1:10010    | ["C","T"]  |       3 |      True | chr1:10010    |
|  1001 | chr1:10011    | ["C","A"]  |       1 |      True | chr1:10011    |
|  1001 | chr1:10011    | ["C","G"]  |       2 |      True | chr1:10011    |
+-------+---------------+------------+---------+-----------+---------------+

+-------------------+-------------------+-------------------+---------------+
| old_alleles       | vep.assembly_name | vep.allele_string | vep.ancestral |
+-------------------+-------------------+-------------------+---------------+
| array<str>        | str               | str               | str           |
+-------------------+-------------------+-------------------+---------------+
| ["C","A","G","T"] | "GRCh38"          | "C/A"             | NA            |
| ["C","A","G","T"] | "GRCh38"          | "C/G"             | NA            |
| ["C","A","G","T"] | "GRCh38"          | "C/T"             | NA            |
| ["C","A","G","T"] | "GRCh38"          | "C/A"             | NA            |
| ["C","A","G","T"] | "GRCh38"          | "C/G"             | NA            |
+-------------------+-------------------+-------------------+---------------+

+------------------------------------------------------------------------------+
| vep.colocated_variants                                                       |
+------------------------------------------------------------------------------+
| array<struct{aa_allele: str, aa_maf: float64, afr_allele: str, afr_maf: f... |
+------------------------------------------------------------------------------+
| NA                                                                           |
| NA                                                                           |
| NA                                                                           |
| NA                                                                           |
| NA                                                                           |
+------------------------------------------------------------------------------+

+-------------+---------+--------+---------------------------+
| vep.context | vep.end | vep.id | vep.input                 |
+-------------+---------+--------+---------------------------+
| str         |   int32 | str    | str                       |
+-------------+---------+--------+---------------------------+
| NA          |   10010 | "."    | "chr1	10010	.	C	A	.	.	GT" |
| NA          |   10010 | "."    | "chr1	10010	.	C	G	.	.	GT" |
| NA          |   10010 | "."    | "chr1	10010	.	C	T	.	.	GT" |
| NA          |   10011 | "."    | "chr1	10011	.	C	A	.	.	GT" |
| NA          |   10011 | "."    | "chr1	10011	.	C	G	.	.	GT" |
+-------------+---------+--------+---------------------------+

+------------------------------------------------------------------------------+
| vep.intergenic_consequences                                                  |
+------------------------------------------------------------------------------+
| array<struct{allele_num: int32, consequence_terms: array<str>, impact: st... |
+------------------------------------------------------------------------------+
| NA                                                                           |
| NA                                                                           |
| NA                                                                           |
| NA                                                       

In [14]:
context_tbl.count()

7939639110

In [29]:
context_sub_tbl = context_tbl.select(
    context_tbl.context, context_tbl.ref, context_tbl.alt,
    context_tbl.variant_type, context_tbl.variant_type_model, 
    context_tbl.methyl_level)

In [30]:
context_sub_tbl.show(5)

,,,,,,,
locus,alleles,context,ref,alt,variant_type,variant_type_model,methyl_level
locus<GRCh38>,array<str>,str,str,str,str,str,int32
chr1:10010,"[""C"",""A""]","""ACC""","""C""","""A""","""transversion""","""non-CpG""",0
chr1:10010,"[""C"",""G""]","""ACC""","""C""","""G""","""transversion""","""non-CpG""",0
chr1:10010,"[""C"",""T""]","""ACC""","""C""","""T""","""non-CpG transition""","""non-CpG""",0
chr1:10011,"[""C"",""A""]","""CCC""","""C""","""A""","""transversion""","""non-CpG""",0
chr1:10011,"[""C"",""G""]","""CCC""","""C""","""G""","""transversion""","""non-CpG""",0


In [31]:
single_context_tbl = context_tbl.filter(context_tbl.context == 'AAA')
single_context_tbl.count()

608914914

In [3]:
608914914/7939639110

0.07669302163029927

In [10]:
genome_tbl = hl.read_table(str(data_dir/'genome_prefiltered.ht'))
genome_tbl.describe()

----------------------------------------
Global fields:
    'freq_meta': array<dict<str, str>> 
    'freq_index_dict': dict<str, int32> 
    'faf_index_dict': dict<str, int32> 
    'faf_meta': array<dict<str, str>> 
    'vep_version': str 
    'vep_csq_header': str 
    'dbsnp_version': str 
    'filtering_model': struct {
        model_name: str, 
        score_name: str, 
        snv_cutoff: struct {
            bin: float64, 
            min_score: float64
        }, 
        indel_cutoff: struct {
            bin: float64, 
            min_score: float64
        }, 
        model_id: str, 
        snv_training_variables: array<str>, 
        indel_training_variables: array<str>
    } 
    'age_distribution': struct {
        bin_edges: array<float64>, 
        bin_freq: array<int32>, 
        n_smaller: int32, 
        n_larger: int32
    } 
----------------------------------------
Row fields:
    'locus': locus<GRCh38> 
    'alleles': array<str> 
    'freq': array<struct {
       

In [11]:
genome_tbl.show(5)

+---------------+------------+
| locus         | alleles    |
+---------------+------------+
| locus<GRCh38> | array<str> |
+---------------+------------+
| chr1:10031    | ["T","C"]  |
| chr1:10037    | ["T","C"]  |
| chr1:10043    | ["T","C"]  |
| chr1:10055    | ["T","C"]  |
| chr1:10057    | ["A","C"]  |
+---------------+------------+

+------------------------------------------------------------------------------+
| freq                                                                         |
+------------------------------------------------------------------------------+
| array<struct{AC: int32, AF: float64, AN: int32, homozygote_count: int32}>    |
+------------------------------------------------------------------------------+
| [(0,0.00e+00,56642,0),(2,1.65e-05,121094,0),(0,0.00e+00,25546,0),(0,0.00e... |
| [(2,2.60e-05,76882,0),(4,3.15e-05,126902,0),(0,0.00e+00,34670,0),(1,1.80e... |
| [(1,1.17e-05,85634,0),(1,8.24e-06,121430,0),(0,0.00e+00,39236,0),(0,0.00e... |
| [(1,1.06e-05,94152,0),(5,4.65e-05,107630,0),(0,0.00e+00,44378,0),(1,1.80e... |
| [(3,2.64e-05,113536,0),(3,2.41e-05,124252,0),(2,3.78e-05,52912,0),(0,0.00... |
+------------------------------------------------------------------------------+

+------------------------------------------------------------------------------+
| raw_qual_hists.gq_hist_all.bin_edges                                         |
+------------------------------------------------------------------------------+
| array<float64>                                                               |
+------------------------------------------------------------------------------+
| [0.00e+00,5.00e+00,1.00e+01,1.50e+01,2.00e+01,2.50e+01,3.00e+01,3.50e+01,... |
| [0.00e+00,5.00e+00,1.00e+01,1.50e+01,2.00e+01,2.50e+01,3.00e+01,3.50e+01,... |
| [0.00e+00,5.00e+00,1.00e+01,1.50e+01,2.00e+01,2.50e+01,3.00e+01,3.50e+01,... |
| [0.00e+00,5.00e+00,1.00e+01,1.50e+01,2.00e+01,2.50e+01,3.00e+01,3.50e+01,... |
| [0.00e+00,5.00e+00,1.00e+01,1.50e+01,2.00e+01,2.50e+01,3.00e+01,3.50e+01,... |
+------------------------------------------------------------------------------+

+------------------------------------------------------------------------------+
| raw_qual_hists.gq_hist_all.bin_freq                                          |
+------------------------------------------------------------------------------+
| array<int64>                                                                 |
+------------------------------------------------------------------------------+
| [14149,7330,6840,3904,23217,2436,1647,594,176,118,49,29,19,9,11,6,5,2,4,2]   |
| [10121,5441,5610,3838,29394,3741,2868,1281,401,358,163,63,69,31,18,20,10,... |
| [6578,3782,4202,3336,31093,4532,3696,1752,595,585,261,88,82,59,17,26,14,5... |
| [1429,1382,1901,2026,25677,6108,5433,3528,1753,1702,1130,485,518,282,119,... |
| [995,1036,1539,1788,25460,7448,6933,4978,2881,2833,2048,1038,1129,685,321... |
+------------------------------------------------------------------------------+

+--------------------------------------+-------------------------------------+
| raw_qual_hists.gq_hist_all.n_smaller | raw_qual_hists.gq_hist_all.n_larger |
+--------------------------------------+-------------------------------------+
|                                int64 |                               int64 |
+--------------------------------------+-------------------------------------+
|                                    0 |                                   0 |
|                                    0 |                                   0 |
|                                    0 |                                   0 |
|                                    0 |                                   0 |
|                                    0 |                                   0 |
+--------------------------------------+-------------------------------------+

+------------------------------------------------------------------------------+
| raw_qual_hists.dp_hist_all.bi

In [12]:
genome_tbl.count()

604162931

- Number of rows of the context table 7,939,639,110
- Number of rows of the genome table 604,162,931